**Loading the Datasets**

In [17]:
import pandas as pd

In [18]:
users = pd.read_csv("Users.csv")
posts = pd.read_csv("Posts.csv")
engagements = pd.read_csv("Engagements.csv")

In [19]:
users.head()

,user_id,age,gender,top_3_interests,past_engagement_score
0,U1,24,F,"sports, art, gaming",0.61
1,U2,32,F,"travel, food, fashion",0.93
2,U3,28,Other,"sports, travel, fashion",0.40
3,U4,25,M,"fashion, music, tech",0.53
4,U5,24,M,"fashion, food, fitness",0.80


In [20]:
posts.head()

,post_id,creator_id,content_type,tags
0,P1,U44,video,"sports, food"
1,P2,U26,video,"music, travel"
2,P3,U32,text,"sports, travel"
3,P4,U6,image,"music, gaming"
4,P5,U32,image,"food, fashion"


In [21]:
engagements.head()

,user_id,post_id,engagement
0,U1,P52,1
1,U1,P44,0
2,U1,P1,1
3,U1,P4,1
4,U1,P65,0


**Checking for Null Values**

In [22]:
users.isnull().sum()

,0
user_id,0
age,0
gender,0
top_3_interests,0
past_engagement_score,0


In [23]:
posts.isnull().sum()

,0
post_id,0
creator_id,0
content_type,0
tags,0


In [24]:
engagements.isnull().sum()

,0
user_id,0
post_id,0
engagement,0


**TF-IDF Vectorization**

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
user_interest_matrix = vectorizer.fit_transform(users["top_3_interests"])
post_tag_matrix = vectorizer.transform(posts["tags"])

**Computing Similarity**

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(user_interest_matrix, post_tag_matrix)

**Engagement Weight**

In [27]:
users["engagement_weight"] = 0.5 + users["past_engagement_score"]

**Generating Recommendations**

In [28]:
recommendations = {}
for i, user_id in enumerate(users["user_id"]):
    # Apply engagement weight to similarity scores
    scores = similarity_matrix[i] * users.loc[i, "engagement_weight"]

    # Get top 3 recommended posts
    top_indices = scores.argsort()[-3:][::-1]
    recommended_posts = posts.iloc[top_indices]["post_id"].tolist()

    recommendations[user_id] = recommended_posts

**Converting to Dataframe**

In [29]:
recommendations_df = pd.DataFrame([
    {"user_id": user, "recommended_posts": recs}
    for user, recs in recommendations.items()
])

In [30]:
recommendations_df.head()

,user_id,recommended_posts
0,U1,"[P78, P22, P87]"
1,U2,"[P42, P80, P5]"
2,U3,"[P97, P3, P34]"
3,U4,"[P37, P53, P96]"
4,U5,"[P74, P7, P26]"


**Saving to CSV**

In [31]:
recommendations_df.to_csv("User_Recommendations.csv", index=False)

**Printing Top 3 Recommendations**

In [32]:
print("Top 3 recommendations generated for each user")
print(recommendations_df.head())

Top 3 recommendations generated for each user
  user_id recommended_posts
0      U1   [P78, P22, P87]
1      U2    [P42, P80, P5]
2      U3    [P97, P3, P34]
3      U4   [P37, P53, P96]
4      U5    [P74, P7, P26]
